# Bài tập 2

Nguyễn Trọng Khiêm - 1712529

## Câu 1 (1 điểm)

In [5]:
import numpy as np
N = 10
mu = 0.5
n_repeats = 100000
n_coins = 1000

nus = np.random.binomial(N, mu, (n_repeats, n_coins)) / N

v1 = nus[:, 0]
vrand = nus[np.arange(nus.shape[0]), np.random.choice(np.arange(nus.shape[1]), size=nus.shape[0])]
vmin = np.min(nus, 1)

print(np.mean(v1), np.mean(vrand), np.mean(vmin))

0.49966900000000003 0.499058 0.037596


Vậy em chọn đáp án [b] 0.01

## Câu 2 (1 điểm)

Theo như tính ở trên, ta thấy mean(v1) và mean(vrand) gần với mu=0.5.  
Do đó em chọn đáp án [d] c1 and crand.

## Câu 3 (1 điểm)

Xác suất h xấp xỉ y là P(x)* P(y|x), bằng (1-muy)* lamda  
==> Xác suất h đồng ý với f tại x và y!=f(x) là (1-muy)* (1-lamda)  
Xác suất h không đồng ý với f tại x nhưng tồn tại y=f(x) là muy* lamda  
Do đó xác suất lỗi mà h tạo ra khi xấp xỉ y là:  
(1-muy)* (1-lamda) + muy* lamda

Vậy em chọn đáp án [e] (1 − λ) ∗ (1 − µ) + λ ∗ µ 

## Câu 4 (1 điểm)

Với λ = 1, khả năng y=f(x) và y!=f(x) là như nhau, nên không phụ thuộc vào m.  
Nếu µ = 1 và có 100% cơ hội h không track f, vẫn có 50% khả năng kết thúc với đúng y.  
Nếu µ = 0 và có 100% cơ hội h track f, vẫn có 50% khả năng kết thúc với sai y.  
  
Vậy em chọn đáp án [b] 0.5

## Câu 5 (1 điểm)

In [6]:
import random
import numpy as np

def generate_target_w():
    """
    Generates target_w (the vector of parameters of f) 
    from two random, uniformly distributed points in [-1, 1] x [-1, 1].
    
    Returns
    -------
    target_w : numpy array, shape (3, 1) 
        The vector of parameters of f.
    """
    # Generate two points from a uniform distribution over [-1, 1]x[-1, 1]
    p1 = np.random.uniform(-1, 1, 2)
    p2 = np.random.uniform(-1, 1, 2)
    # Compute the target W from these two points
    target_w = np.array([p1[1]*p2[0] - p1[0]*p2[1], p2[1] - p1[1], p1[0] - p2[0]]).reshape((-1, 1))
    
    return target_w
def generate_data(N, target_w):
    """
    Generates a data set by generating random inputs and then using target_w to generate the 
    corresponding outputs.
    
    Parameters
    ----------
    N : int
        The number of examples.
    target_w : numpy array, shape (3, 1) 
        The vector of parameters of f.
    
    Returns
    -------
    X : numpy array, shape (N, 3)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    Y : numpy array, shape (N, 1)
        The vector of outputs.        
    """
    bad_data = True # `bad_data = True` means: data contain points on the target line 
                    # (this rarely happens, but just to be careful)
                    # -> y's of these points = 0 (with np.sign); 
                    #    we don't want this (y's of data must be -1 or 1)
                    # -> re-generate data until `bad_data = False`
    
    while bad_data == True:
        X = np.random.uniform(-1, 1, (N, 2))
        X = np.hstack((np.ones((N, 1)), X)) # Add 'ones' column
        Y = np.sign(np.dot(X, target_w))
        if (0 not in Y): # Good data
            bad_data = False
    
    return X, Y



def train_LR(X, ys):
    X_dagger = np.dot(np.linalg.pinv(np.dot(X.T, X)), X.T)
    w = np.dot(X_dagger, ys)
    return w



def main(N_in, N_out):
    num_runs = 1000
    avg_E_in = 0.0
    avg_E_out = 0.0
    for i in range(num_runs):
        # Generate target_w
        target_w = generate_target_w()

        # Generate training set
        X, Y = generate_data(N_in, target_w)

        # Generate test set
        X_test, Y_test = generate_data(N_out, target_w)

        w = train_LR(X, Y)

        predictions_in = np.dot(X, w)

        E_in = np.mean(np.sign(predictions_in) != Y)
        avg_E_in += (E_in * 1.0 / num_runs)
        
        
        target_w_out = generate_target_w()
        
        
        predictions_out = np.dot(X_test, w)
        E_out = np.mean(np.sign(predictions_out) != Y_test)
        avg_E_out += (E_out * 1.0 / num_runs)

    print('avg_E_in = %f' % (avg_E_in))
    print('avg_E_out = %f' % (avg_E_out))
    
main(100, 1000)

avg_E_in = 0.039440
avg_E_out = 0.048060


Kết quả gần với 0.01 nhất.  
Vậy em chọn đáp án [c] 0.01

## Câu 6 (1 điểm)

Kết quả gần với 0.01 nhất.  
Vậy em chọn đáp án [c] 0.01

## Câu 7 (1 điểm)

In [7]:
def train_PLA(X, ys):
    w = train_LR(X, ys)
    n_iterations = 0
    while True:
        predictions = np.sign(np.dot(X, w))
        
        errs = predictions!=ys
        #print(errs.shape,errs)
        mean_err = np.mean(errs)
        if mean_err!=0:
            
            rand_err_idx = np.random.choice(np.flatnonzero(errs),1)
            w = w + X[rand_err_idx].transpose()*ys[rand_err_idx]
            
            n_iterations+=1
        else:
            break
    return w, n_iterations
def cau7(N):
    num_runs = 1000
    avg_n_iters = 0.0
    for r in range(num_runs):
        # Generate target_w
        target_w = generate_target_w()
        
        # Generate training set
        X, Y = generate_data(N, target_w)
        
        
        w, n_iters = train_PLA(X, Y)
        
        avg_n_iters += (n_iters * 1.0 / num_runs)
    print('avg_num_iterations = %f' % (avg_n_iters))



In [8]:
cau7(10)

avg_num_iterations = 3.920000


Kết quả gần với 1 nhất.  
Vậy em chọn đáp án [a] 1

## Câu 8 (1 điểm)

In [8]:
import numpy as np
a=np.array([[1,2],[3,4]])
print(a**2)
b=np.array([[6],[2]])
np.dot(a**2,b)

[[ 1  4]
 [ 9 16]]


array([[14],
       [86]])

In [ ]:
def generate_data(N, target_w):
    bad_data = True # `bad_data = True` means: data contain points on the target line 
                    # (this rarely happens, but just to be careful)
                    # -> y's of these points = 0 (with np.sign); 
                    #    we don't want this (y's of data must be -1 or 1)
                    # -> re-generate data until `bad_data = False`
    
    while bad_data == True:
        X = np.random.uniform(-1, 1, (N, 2))
        X = np.hstack((np.ones((N, 1)), X)) # Add 'ones' column
        Y = np.sign(np.dot(X**2, target_w))
        if (0 not in Y): # Good data
            bad_data = False
    
    return X, Y

def transform(X):
    
def main(N):
    num_runs = 1000
    avg_E_in = 0.0
    for i in range(num_runs):
        # Generate target_w
        target_w = np.array([[1.0],[1.0],[-0.6]])

        # Generate training set
        X, Y = generate_data(N, target_w)

        idx = np.random.choice(np.arange(N), size=int(0.1*N), replace=False)
        
        for x in idx:
            Y[x]=-Y[x]
        

YOUR ANSWER HERE

## Câu 9 (1 điểm)

YOUR ANSWER HERE

## Câu 10 (1 điểm)

YOUR ANSWER HERE